# Kapitola 22: Zpracovani dat - Kucharks pro pripravu AI

## Jak pripravit data, aby AI mohla ucit

Predstavte si, ze jste sefkuchar a prave vam dorazila bedna se zeleninou. Jsou v ni krasne mrkve, ale i par nahnilych. Nektere brambory jsou obri, jine malicke. A par kousku zeleniny vubec nepoznavate.

**Co udelate?** Urcite nezacnete hned varit! Nejprve si suroviny **pripravite**:
- Vyhodite shnile kusy
- Oloupete brambory
- Nakrajite vse na stejnou velikost

Presne totez musime udelat s daty pro AI. Tomuto procesu se rika **cisteni a predzpracovani dat**.

---

### Co se naucime:
1. **Duplikaty** - jak najit a odstranit zdvojene zaznamy
2. **Chybejici hodnoty** - co delat, kdyz nam nejaka data chybi
3. **Odlehle hodnoty (Outliers)** - jak poznat a ricit s extremnimi daty
4. **Normalizace** - jak sjednotit meritka ruznych dat
5. **Vizualizace cisteni** - jak videt, co jsme s daty udelali

### Proc je to dulezite?
V datove vede plati zlate pravidlo: **"Garbage in, garbage out"** (Odpadky dovnitr, odpadky ven). I ten nejlepsi AI model selze, pokud dostane spatna data!

## 1. Instalace a import knihoven

Budeme potrebovat:
- **Pandas** - prace s tabulkovymi daty
- **NumPy** - matematicke operace
- **Matplotlib/Seaborn** - vizualizace
- **Scikit-learn** - nastroje pro normalizaci

In [ ]:
# Instalace knihoven (pro Google Colab)
!pip install pandas numpy matplotlib seaborn scikit-learn -q

print("Knihovny nainstalovany!")

In [ ]:
# Import knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Nastaveni vizualizaci
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Vsechny knihovny uspesne nacteny!")
print(f"Pandas verze: {pd.__version__}")
print(f"NumPy verze: {np.__version__}")

## 2. Nas "spinavy" dataset: Fiktivni inzeraty na byty

Vytvorime si maly, ale zamerne "spinavy" dataset s nabidkami bytu.

**Bude obsahovat typicke problemy realnych dat:**
- Duplicitni zaznamy (stejne ID)
- Chybejici hodnoty (prazdna pole)
- Odlehle hodnoty (extremne vysoka cena)
- Ruzne jednotky a meritka

In [ ]:
# Vytvorime zamerne "spinavy" dataset
data = {
    'id': [1, 2, 3, 4, 5, 5, 6, 7, 8, 9],  # Pozor: ID 5 je dvakrat!
    'lokalita': ['Praha', 'Brno', 'Praha', 'Ostrava', 'Praha', 
                 'Praha', 'Brno', 'Plzen', 'Praha', 'Brno'],
    'plocha_m2': [60, 75, 55, 120, 65, 65, 500, 70, 45, np.nan],  # 500 je podezrele, jedno chybi
    'cena_mil_kc': [5.5, 6.8, 5.1, 11.0, 6.1, 6.1, 50.0, 6.5, 4.2, 5.8],  # 50 mil je extremni
    'patro': [3, 5, 2, 8, np.nan, 4, 1, 2, 7, 3],  # Jedno patro chybi
    'rok_vystavby': [1980, 2010, 1975, 2020, 1990, 1990, 2015, 1965, 2005, np.nan]
}

byty_df = pd.DataFrame(data)

print("=" * 60)
print("PUVODNI 'SPINAVA' DATA")
print("=" * 60)
print()
print(byty_df.to_string())
print()

# Zakladni informace o datasetu
print("\n" + "=" * 60)
print("ZAKLADNI INFORMACE O DATASETU")
print("=" * 60)
byty_df.info()

In [ ]:
# Vizualizace puvodnich dat
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Histogram plochy
axes[0].hist(byty_df['plocha_m2'].dropna(), bins=10, color='coral', edgecolor='black')
axes[0].set_xlabel('Plocha (m2)')
axes[0].set_ylabel('Pocet')
axes[0].set_title('Rozlozeni plochy bytu')
axes[0].axvline(x=500, color='red', linestyle='--', label='Outlier (500 m2)')
axes[0].legend()

# Histogram ceny
axes[1].hist(byty_df['cena_mil_kc'].dropna(), bins=10, color='skyblue', edgecolor='black')
axes[1].set_xlabel('Cena (mil. Kc)')
axes[1].set_ylabel('Pocet')
axes[1].set_title('Rozlozeni cen bytu')
axes[1].axvline(x=50, color='red', linestyle='--', label='Outlier (50 mil)')
axes[1].legend()

# Boxplot pro detekci outlieru
axes[2].boxplot([byty_df['plocha_m2'].dropna(), byty_df['cena_mil_kc'].dropna()],
                labels=['Plocha (m2)', 'Cena (mil)'])
axes[2].set_title('Boxplot - detekce outlieru')

plt.suptitle('Analyza puvodnich "spinavych" dat', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCervene cary oznacuji extremni hodnoty (outliers)!")

## 3. Krok 1: Odstraneni duplikatu

**Duplicitni zaznamy** mohou zkreslit statistiky a uceni modelu.

V nasem datasetu je ID 5 dvakrat - to je chyba, kterou musime opravit.

In [ ]:
# Detekce duplikatu
print("=" * 60)
print("DETEKCE DUPLIKATU")
print("=" * 60)

# Najdeme duplikaty podle ID
duplikaty = byty_df[byty_df['id'].duplicated(keep=False)]
print("\nDuplicitni zaznamy (stejne ID):")
print(duplikaty)

pocet_duplikatu = byty_df['id'].duplicated().sum()
print(f"\nCelkem nalezeno duplicitnich ID: {pocet_duplikatu}")

In [ ]:
# Odstraneni duplikatu - ponechame prvni vyskyt
byty_bez_duplikatu = byty_df.drop_duplicates(subset='id', keep='first')

print("=" * 60)
print("DATA PO ODSTRANENI DUPLIKATU")
print("=" * 60)
print(f"\nPuvodni pocet radku: {len(byty_df)}")
print(f"Pocet radku po odstraneni duplikatu: {len(byty_bez_duplikatu)}")
print(f"Odstraneno radku: {len(byty_df) - len(byty_bez_duplikatu)}")
print()
print(byty_bez_duplikatu.to_string())

## 4. Krok 2: Zpracovani chybejicich hodnot

V realnych datech casto **nektere hodnoty chybi**. Mame nekolik strategii:

| Strategie | Popis | Kdy pouzit |
|-----------|-------|------------|
| **Smazani** (dropna) | Odstranime cely radek | Kdyz chybi malo dat a radku mame dost |
| **Median** | Nahradime prostredni hodnotou | Pro cisla, kdyz jsou outliers |
| **Prumer** (mean) | Nahradime prumerem | Pro cisla s normalnim rozlozenim |
| **Modus** | Nejcastejsi hodnota | Pro kategoricka data |

In [ ]:
# Analyza chybejicich hodnot
print("=" * 60)
print("ANALYZA CHYBEJICICH HODNOT")
print("=" * 60)

# Pocet chybejicich hodnot v kazdem sloupci
chybejici = byty_bez_duplikatu.isnull().sum()
print("\nPocet chybejicich hodnot v kazdem sloupci:")
print(chybejici)

# Procento chybejicich hodnot
print("\nProcento chybejicich hodnot:")
procenta = (chybejici / len(byty_bez_duplikatu) * 100).round(1)
for col, proc in procenta.items():
    if proc > 0:
        print(f"  {col}: {proc}%")

In [ ]:
# Vizualizace chybejicich hodnot
plt.figure(figsize=(10, 4))

# Heatmapa chybejicich hodnot
sns.heatmap(byty_bez_duplikatu.isnull(), cbar=True, cmap='YlOrRd', 
            yticklabels=byty_bez_duplikatu['id'].values)
plt.title('Mapa chybejicich hodnot (zluta = chybi)', fontsize=14)
plt.xlabel('Sloupce')
plt.ylabel('ID bytu')
plt.tight_layout()
plt.show()

In [ ]:
# Vytvorime kopii pro praci
byty_cisteni = byty_bez_duplikatu.copy()

print("=" * 60)
print("DOPLNENI CHYBEJICICH HODNOT")
print("=" * 60)

# 1. Plocha - doplnime MEDIANEM (robustnejsi vuci outliers)
median_plocha = byty_cisteni['plocha_m2'].median()
byty_cisteni['plocha_m2'].fillna(median_plocha, inplace=True)
print(f"\nPlocha - doplneno medianem: {median_plocha} m2")

# 2. Patro - doplnime MEDIANEM
median_patro = byty_cisteni['patro'].median()
byty_cisteni['patro'].fillna(median_patro, inplace=True)
print(f"Patro - doplneno medianem: {median_patro}")

# 3. Rok vystavby - doplnime PRUMEREM (zaokrouhleno)
prumer_rok = round(byty_cisteni['rok_vystavby'].mean())
byty_cisteni['rok_vystavby'].fillna(prumer_rok, inplace=True)
print(f"Rok vystavby - doplneno prumerem: {prumer_rok}")

print("\nData po doplneni chybejicich hodnot:")
print(byty_cisteni.to_string())

In [ ]:
# Overeni - uz zadne chybejici hodnoty
print("\nOvereni - pocet chybejicich hodnot:")
print(byty_cisteni.isnull().sum())
print("\n" + "="*40)
if byty_cisteni.isnull().sum().sum() == 0:
    print("USPECH! Zadne chybejici hodnoty.")
else:
    print("VAROVANI: Stale jsou chybejici hodnoty!")

## 5. Krok 3: Odstraneni odlehlych hodnot (Outliers)

**Odlehle hodnoty** jsou extremni data, ktera mohou zkreslit uceni modelu.

V nasem datasetu:
- Byt s plochou **500 m2** a cenou **50 mil. Kc** je zjevne "mimo"
- Muze to byt chyba v datech nebo luxusni penthouse

**Metody detekce outlieru:**
1. **IQR metoda** - hodnoty mimo 1.5 * mezikvartilove rozpeti
2. **Z-score** - hodnoty vice nez 3 smerodatne odchylky od prumeru
3. **Domenova znalost** - vime, ze normalni byt nema 500 m2

In [ ]:
# Vizualizace outlieru pomoci boxplotu
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Boxplot plochy
bp1 = axes[0].boxplot(byty_cisteni['plocha_m2'], patch_artist=True)
bp1['boxes'][0].set_facecolor('lightcoral')
axes[0].set_ylabel('Plocha (m2)')
axes[0].set_title('Boxplot: Plocha bytu')
axes[0].axhline(y=500, color='red', linestyle='--', label='Outlier: 500 m2')
axes[0].legend()

# Boxplot ceny
bp2 = axes[1].boxplot(byty_cisteni['cena_mil_kc'], patch_artist=True)
bp2['boxes'][0].set_facecolor('lightblue')
axes[1].set_ylabel('Cena (mil. Kc)')
axes[1].set_title('Boxplot: Cena bytu')
axes[1].axhline(y=50, color='red', linestyle='--', label='Outlier: 50 mil')
axes[1].legend()

plt.suptitle('Detekce outlieru pomoci boxplotu', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Body mimo 'vousy' boxplotu jsou potencialni outliers!")

In [ ]:
# IQR metoda pro detekci outlieru
def detekuj_outliers_iqr(data, sloupec):
    """Detekuje outliers pomoci IQR metody"""
    Q1 = data[sloupec].quantile(0.25)
    Q3 = data[sloupec].quantile(0.75)
    IQR = Q3 - Q1
    
    dolni_hranice = Q1 - 1.5 * IQR
    horni_hranice = Q3 + 1.5 * IQR
    
    outliers = data[(data[sloupec] < dolni_hranice) | (data[sloupec] > horni_hranice)]
    
    return outliers, dolni_hranice, horni_hranice

print("=" * 60)
print("DETEKCE OUTLIERU POMOCI IQR METODY")
print("=" * 60)

# Outliers v plose
outliers_plocha, dol_p, hor_p = detekuj_outliers_iqr(byty_cisteni, 'plocha_m2')
print(f"\nPlocha - normalni rozsah: {dol_p:.1f} - {hor_p:.1f} m2")
print(f"Nalezeno outlieru: {len(outliers_plocha)}")
if len(outliers_plocha) > 0:
    print(outliers_plocha[['id', 'lokalita', 'plocha_m2', 'cena_mil_kc']])

# Outliers v cene
outliers_cena, dol_c, hor_c = detekuj_outliers_iqr(byty_cisteni, 'cena_mil_kc')
print(f"\nCena - normalni rozsah: {dol_c:.1f} - {hor_c:.1f} mil. Kc")
print(f"Nalezeno outlieru: {len(outliers_cena)}")
if len(outliers_cena) > 0:
    print(outliers_cena[['id', 'lokalita', 'plocha_m2', 'cena_mil_kc']])

In [ ]:
# Odstraneni outlieru
print("=" * 60)
print("ODSTRANENI OUTLIERU")
print("=" * 60)

# Pouzijeme jednoduchou podminkovou filtraci
# Odstranime byty s cenou vyssi nez 20 mil nebo plochou vetsi nez 200 m2
byty_bez_outlieru = byty_cisteni[
    (byty_cisteni['cena_mil_kc'] < 20) & 
    (byty_cisteni['plocha_m2'] < 200)
]

print(f"\nPuvodni pocet radku: {len(byty_cisteni)}")
print(f"Pocet radku po odstraneni outlieru: {len(byty_bez_outlieru)}")
print(f"Odstraneno outlieru: {len(byty_cisteni) - len(byty_bez_outlieru)}")
print()
print("Data bez outlieru:")
print(byty_bez_outlieru.to_string())

In [ ]:
# Porovnani pred a po odstraneni outlieru
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot PRED
axes[0].scatter(byty_cisteni['plocha_m2'], byty_cisteni['cena_mil_kc'], 
                c='coral', s=100, alpha=0.7, edgecolors='black')
axes[0].set_xlabel('Plocha (m2)')
axes[0].set_ylabel('Cena (mil. Kc)')
axes[0].set_title('PRED odstranenim outlieru')
# Oznacime outlier
outlier = byty_cisteni[byty_cisteni['plocha_m2'] > 200]
for _, row in outlier.iterrows():
    axes[0].annotate('OUTLIER!', (row['plocha_m2'], row['cena_mil_kc']),
                     fontsize=10, color='red', fontweight='bold')

# Scatter plot PO
axes[1].scatter(byty_bez_outlieru['plocha_m2'], byty_bez_outlieru['cena_mil_kc'], 
                c='lightgreen', s=100, alpha=0.7, edgecolors='black')
axes[1].set_xlabel('Plocha (m2)')
axes[1].set_ylabel('Cena (mil. Kc)')
axes[1].set_title('PO odstraneni outlieru')

plt.suptitle('Porovnani dat pred a po odstraneni outlieru', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Krok 4: Normalizace dat

**Problem:** Ruzne sloupce maji ruzna meritka:
- `plocha_m2` je v desitkach (45-120)
- `cena_mil_kc` je v jednotkach (4-11)
- `rok_vystavby` je v tisicich (1965-2020)

AI modely by prikladaly vetsi vahu plose jen proto, ze je to vetsi cislo!

**Reseni - Normalizace:**

| Metoda | Vzorec | Rozsah | Kdy pouzit |
|--------|--------|--------|------------|
| **Min-Max** | (x - min) / (max - min) | 0 az 1 | Kdyz chceme pevny rozsah |
| **Z-score** | (x - mean) / std | -3 az +3 | Kdyz chceme zachovat rozlozeni |

In [ ]:
# Statistiky pred normalizaci
print("=" * 60)
print("STATISTIKY PRED NORMALIZACI")
print("=" * 60)

# Vybereme numericke sloupce pro normalizaci
sloupce_k_normalizaci = ['plocha_m2', 'cena_mil_kc', 'patro', 'rok_vystavby']

print("\nPuvodni hodnoty:")
print(byty_bez_outlieru[sloupce_k_normalizaci].describe().round(2))

In [ ]:
# Min-Max normalizace
print("=" * 60)
print("MIN-MAX NORMALIZACE")
print("=" * 60)

# Vytvorime scaler
minmax_scaler = MinMaxScaler()

# Ziskame data k normalizaci
data_k_normalizaci = byty_bez_outlieru[sloupce_k_normalizaci].values

# Aplikujeme normalizaci
normalizovana_data_minmax = minmax_scaler.fit_transform(data_k_normalizaci)

# Vytvorime novy DataFrame
byty_minmax = pd.DataFrame(
    normalizovana_data_minmax, 
    columns=[f"{col}_norm" for col in sloupce_k_normalizaci]
)

print("\nNormalizovana data (Min-Max, rozsah 0-1):")
print(byty_minmax.round(3).to_string())

print("\nStatistiky po Min-Max normalizaci:")
print(byty_minmax.describe().round(3))

In [ ]:
# Z-score standardizace
print("=" * 60)
print("Z-SCORE STANDARDIZACE")
print("=" * 60)

# Vytvorime scaler
zscore_scaler = StandardScaler()

# Aplikujeme standardizaci
standardizovana_data = zscore_scaler.fit_transform(data_k_normalizaci)

# Vytvorime novy DataFrame
byty_zscore = pd.DataFrame(
    standardizovana_data, 
    columns=[f"{col}_std" for col in sloupce_k_normalizaci]
)

print("\nStandardizovana data (Z-score, prumer=0, std=1):")
print(byty_zscore.round(3).to_string())

print("\nStatistiky po Z-score standardizaci:")
print(byty_zscore.describe().round(3))

In [ ]:
# Vizualizace porovnani normalizacnich metod
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Puvodni data
axes[0].bar(range(len(sloupce_k_normalizaci)), 
            byty_bez_outlieru[sloupce_k_normalizaci].mean(),
            color=['coral', 'skyblue', 'lightgreen', 'plum'])
axes[0].set_xticks(range(len(sloupce_k_normalizaci)))
axes[0].set_xticklabels(['plocha', 'cena', 'patro', 'rok'], rotation=45)
axes[0].set_title('PUVODNI prumery')
axes[0].set_ylabel('Hodnota')

# Min-Max
axes[1].bar(range(len(sloupce_k_normalizaci)), 
            byty_minmax.mean(),
            color=['coral', 'skyblue', 'lightgreen', 'plum'])
axes[1].set_xticks(range(len(sloupce_k_normalizaci)))
axes[1].set_xticklabels(['plocha', 'cena', 'patro', 'rok'], rotation=45)
axes[1].set_title('MIN-MAX prumery (0-1)')
axes[1].set_ylabel('Hodnota')
axes[1].set_ylim(0, 1)

# Z-score
axes[2].bar(range(len(sloupce_k_normalizaci)), 
            byty_zscore.mean(),
            color=['coral', 'skyblue', 'lightgreen', 'plum'])
axes[2].set_xticks(range(len(sloupce_k_normalizaci)))
axes[2].set_xticklabels(['plocha', 'cena', 'patro', 'rok'], rotation=45)
axes[2].set_title('Z-SCORE prumery (~0)')
axes[2].set_ylabel('Hodnota')
axes[2].axhline(y=0, color='red', linestyle='--')

plt.suptitle('Porovnani normalizacnich metod', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Finalni vysledek: Cista data pripravena pro AI

Podivejme se na celkovy vysledek naseho cisteni!

In [ ]:
# Finalni prehled
print("=" * 70)
print("SOUHRN CISTENI DAT")
print("=" * 70)

print("\n1. PUVODNI DATA:")
print(f"   - Pocet radku: {len(byty_df)}")
print(f"   - Duplikaty: ANO (ID 5 dvakrat)")
print(f"   - Chybejici hodnoty: ANO (plocha, patro, rok)")
print(f"   - Outliers: ANO (500 m2, 50 mil Kc)")

print("\n2. PO ODSTRANENI DUPLIKATU:")
print(f"   - Pocet radku: {len(byty_bez_duplikatu)}")
print(f"   - Odstraneno: {len(byty_df) - len(byty_bez_duplikatu)} radku")

print("\n3. PO DOPLNENI CHYBEJICICH HODNOT:")
print(f"   - Metoda: Median pro plochu/patro, Prumer pro rok")
print(f"   - Chybejici hodnoty: 0")

print("\n4. PO ODSTRANENI OUTLIERU:")
print(f"   - Pocet radku: {len(byty_bez_outlieru)}")
print(f"   - Odstraneno: {len(byty_cisteni) - len(byty_bez_outlieru)} outlieru")

print("\n5. PO NORMALIZACI:")
print(f"   - Min-Max: vsechny hodnoty v rozsahu 0-1")
print(f"   - Z-score: prumer=0, smerodatna odchylka=1")

print("\n" + "=" * 70)
print("DATA JSOU PRIPRAVENA PRO TRENOVANI AI MODELU!")
print("=" * 70)

In [ ]:
# Vytvorime finalni dataset
finalni_data = byty_bez_outlieru.copy()
finalni_data = finalni_data.reset_index(drop=True)

# Pridame normalizovane sloupce
for i, col in enumerate(sloupce_k_normalizaci):
    finalni_data[f"{col}_norm"] = byty_minmax.iloc[:, i].values

print("FINALNI CISTY DATASET:")
print()
print(finalni_data.to_string())

## 8. Mini-projekt: Cisteni realneho datasetu

Aplikujme vse, co jsme se naucili, na vetsi dataset - data o pasazerech Titanicu!

In [ ]:
# Nacteme Titanic dataset z internetu
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
titanic = pd.read_csv(url)

print("=" * 60)
print("TITANIC DATASET - PUVODNI STAV")
print("=" * 60)

print(f"\nPocet radku: {len(titanic)}")
print(f"Pocet sloupcu: {len(titanic.columns)}")
print("\nPrvnich 5 radku:")
print(titanic.head())

In [ ]:
# Analyza chybejicich hodnot
print("=" * 60)
print("ANALYZA CHYBEJICICH HODNOT")
print("=" * 60)

chybejici = titanic.isnull().sum()
chybejici_procent = (chybejici / len(titanic) * 100).round(1)

print("\nChybejici hodnoty:")
for col in titanic.columns:
    if chybejici[col] > 0:
        print(f"  {col}: {chybejici[col]} ({chybejici_procent[col]}%)")

In [ ]:
# Cisteni Titanic datasetu
print("=" * 60)
print("CISTENI TITANIC DATASETU")
print("=" * 60)

titanic_cisty = titanic.copy()

# 1. Age - doplnime medianem
median_age = titanic_cisty['Age'].median()
titanic_cisty['Age'].fillna(median_age, inplace=True)
print(f"\n1. Age - doplneno medianem: {median_age}")

# 2. Embarked - doplnime modem (nejcastejsi hodnota)
modus_embarked = titanic_cisty['Embarked'].mode()[0]
titanic_cisty['Embarked'].fillna(modus_embarked, inplace=True)
print(f"2. Embarked - doplneno modem: {modus_embarked}")

# 3. Cabin - prilis chybi (77%), odstranime sloupec
titanic_cisty = titanic_cisty.drop('Cabin', axis=1)
print(f"3. Cabin - sloupec odstranen (77% chybi)")

# 4. Odstraneni duplikatu podle PassengerId
pred = len(titanic_cisty)
titanic_cisty = titanic_cisty.drop_duplicates(subset='PassengerId')
print(f"\n4. Odstraneno duplikatu: {pred - len(titanic_cisty)}")

# 5. Overeni
print(f"\nChybejici hodnoty po cisteni: {titanic_cisty.isnull().sum().sum()}")

In [ ]:
# Normalizace numerickych sloupcu
print("=" * 60)
print("NORMALIZACE TITANICU")
print("=" * 60)

numericke_sloupce = ['Age', 'Fare', 'SibSp', 'Parch']

scaler = MinMaxScaler()
titanic_norm = scaler.fit_transform(titanic_cisty[numericke_sloupce])

print("\nPuvodni statistiky:")
print(titanic_cisty[numericke_sloupce].describe().round(2))

print("\nNormalizovane statistiky (Min-Max):")
titanic_norm_df = pd.DataFrame(titanic_norm, columns=numericke_sloupce)
print(titanic_norm_df.describe().round(3))

In [ ]:
# Vizualizace vysledku
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Rozlozeni veku
axes[0, 0].hist(titanic_cisty['Age'], bins=30, color='skyblue', edgecolor='black')
axes[0, 0].axvline(x=median_age, color='red', linestyle='--', label=f'Median: {median_age}')
axes[0, 0].set_xlabel('Vek')
axes[0, 0].set_ylabel('Pocet')
axes[0, 0].set_title('Rozlozeni veku pasazeru')
axes[0, 0].legend()

# Rozlozeni jizdenek
axes[0, 1].hist(titanic_cisty['Fare'], bins=30, color='lightcoral', edgecolor='black')
axes[0, 1].set_xlabel('Cena jizdenky')
axes[0, 1].set_ylabel('Pocet')
axes[0, 1].set_title('Rozlozeni cen jizdenek')

# Boxplot veku podle preziti
titanic_cisty.boxplot(column='Age', by='Survived', ax=axes[1, 0])
axes[1, 0].set_xlabel('Prezil (0=Ne, 1=Ano)')
axes[1, 0].set_ylabel('Vek')
axes[1, 0].set_title('Vek podle preziti')
plt.suptitle('')  # Odstranime automaticky nadpis

# Pristav nalodeni
titanic_cisty['Embarked'].value_counts().plot(kind='bar', ax=axes[1, 1], color='lightgreen', edgecolor='black')
axes[1, 1].set_xlabel('Pristav')
axes[1, 1].set_ylabel('Pocet pasazeru')
axes[1, 1].set_title('Pocet pasazeru podle pristavu')
axes[1, 1].set_xticklabels(['Southampton', 'Cherbourg', 'Queenstown'], rotation=0)

plt.suptitle('Titanic dataset po vycisteni', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 9. Shruti a kontrolni otazky

### Co jsme se naucili:

1. **Duplikaty** - `drop_duplicates()` odstarani zdvojene zaznamy
2. **Chybejici hodnoty** - `fillna()` doplni median, prumer nebo modus
3. **Outliers** - IQR metoda nebo domenova znalost je identifikuje
4. **Normalizace** - `MinMaxScaler` (0-1) nebo `StandardScaler` (z-score)

### Kontrolni otazky:

1. **Proc je dulezite odstranit duplikaty pred trenovanim modelu?**
2. **Jaky je rozdil mezi medianem a prumerem pri doplnovani chybejicich hodnot?**
3. **Kdy byste pouzili Min-Max normalizaci a kdy Z-score?**
4. **Co znamena "Garbage in, garbage out"?**

In [ ]:
# Test vasich znalosti
print("=" * 60)
print("KVIZ: TEST VASICH ZNALOSTI")
print("=" * 60)

otazky = [
    {
        "otazka": "Ktera metoda je robustnejsi vuci outliers?",
        "moznosti": ["A) Prumer", "B) Median", "C) Modus"],
        "spravne": "B"
    },
    {
        "otazka": "Jaky rozsah hodnot vytvari Min-Max normalizace?",
        "moznosti": ["A) -1 az 1", "B) 0 az 100", "C) 0 az 1"],
        "spravne": "C"
    },
    {
        "otazka": "Co je outlier?",
        "moznosti": ["A) Chybejici hodnota", "B) Extremni hodnota", "C) Duplikat"],
        "spravne": "B"
    }
]

spravne = 0
for i, q in enumerate(otazky, 1):
    print(f"\nOtazka {i}: {q['otazka']}")
    for m in q['moznosti']:
        print(f"  {m}")
    print(f"  --> Spravna odpoved: {q['spravne']}")
    spravne += 1

print(f"\n" + "="*40)
print(f"Spravne odpovedi: {spravne}/{len(otazky)}")

## 10. Vase vyzva

Zkuste v nasem kodu jinou strategii:

1. **Misto medianu pouzijte prumer** - jak se zmeni vysledky?
2. **Smazte radky s chybejicimi hodnotami** - kolik dat ztratite?
3. **Najdete jiny dataset** a aplikujte na nej cisteni

```python
# Tip: Zkuste tyto datasety
# Iris: https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
# Wine: https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data
```

In [ ]:
# Vas prostor pro experimentovani
# -----------------------------

# Zkuste zmenit zpusob doplneni chybejicich hodnot:
# byty_cisteni['plocha_m2'].fillna(byty_cisteni['plocha_m2'].mean(), inplace=True)

# Nebo zkuste smazat radky s chybejicimi hodnotami:
# byty_cisteni = byty_cisteni.dropna()

print("Experimentujte s kodem vyse!")